# 3.1 Tensorflow计算模型——计算图

## 3.1.1 计算图的概念

## 3.1.2 计算图的使用
TensorFlow程序一般可以分为两个阶段：
1. 定义计算图汇总所有的计算；
2. 执行计算。

In [7]:
import tensorflow as tf

In [9]:
a = tf.constant([1.0, 2.0], name='a')
b = tf.constant([2.0, 3.0], name='b')
res = a + b

In [10]:
print(a.graph is tf.get_default_graph())

True


In [24]:
g1 = tf.Graph()
with g1.as_default():
    v = tf.get_variable("v", initializer=tf.zeros_initializer(shape=[1]))

g2 = tf.Graph()
with g2.as_default():
    v = tf.get_variable('v', shape=[1], initializer=tf.ones_initializer())
    
with tf.Session(graph=g1) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable("v")))
        
with tf.Session(graph=g2) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable("v")))

[0.]
[1.]


上面的代码产生了两个计算图，每个计算图中定义了一个名字为“v”的变量。可以看到当运行不同计算图时，
变量v的值也是不一样的。TensorFlow中的计算图可以用来隔离张量和计算，它还提供了管理张量和计算的机制。

In [27]:
# 计算图可以通过tf.Graph.device函数来指定运行计算的设备
g = tf.Graph()
with g.device('/gpu:0'):
    res = a + b

# 3.2 TensorFlow数据模型——张量

## 3.2.1 张量的概念

In [28]:
a = tf.constant([1.0, 2.0], name='a')
b = tf.constant([2.0, 3.0], name='b')
result = tf.add(a, b, name='add')
print(result)

Tensor("add_4:0", shape=(2,), dtype=float32)


一个张量中主要保存了三个属性：名字、维度和类型

## 3.2.2 张量的使用

1. 第一类用途是对中间计算结果的引用。
2. 张量可以用来获得计算的结果。

# 3.4 TensorFlow实现神经网络

In [35]:
w1 = tf.Variable(tf.random_normal([2, 3], stddev=1))
w2 = tf.Variable(tf.random_normal([3, 1], stddev=1))

x = tf.placeholder(tf.float32, shape=(None,2), name='input')
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)

print(sess.run(y, feed_dict={x:[[0.7, 0.9]]}))
print(sess.run(y, feed_dict={x:[[0.7, 0.9],[0.1, 0.4],[0.5, 0.8]]}))

[[-0.2649628]]
[[-0.2649628 ]
 [-0.05774263]
 [-0.20077479]]


In [37]:
# 完整神经网络样例程序
# 下面给出一个完整的程序来训练神经网络解决二分类问题
from numpy.random import RandomState

In [38]:
batch_size = 8

w1 = tf.Variable(tf.random_normal([2, 3], stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal([3, 1], stddev=1, seed=1))

x = tf.placeholder(shape=(None, 2), dtype=tf.float32, name='x-input')
y_ = tf.placeholder(shape=(None, 1), dtype=tf.float32, name='y-input')

a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

cross_entropy = -tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y, 1e-10, 1.0)))
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

# 通过随机数生成模拟数据集
rdm = RandomState(1)
dataset_size = 128
X = rdm.rand(dataset_size, 2)
Y = [[int(x1+x2 < 1)] for (x1, x2) in X]

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('训练之前参数值：')
    print('w1:',w1)
    print('w2:',w2)
    STEPS = 5000
    for i in range(STEPS):
        start = (i * batch_size) % dataset_size
        end = min(start+batch_size, dataset_size)
        sess.run(train_step, feed_dict={x: X[start:end], y_:Y[start:end]})
        if i % 1000 == 0:
            total_cross_entropy = sess.run(cross_entropy, feed_dict={x:X, y_:Y})
            print('After %d trainnig step(s), croos entropy on all data is %g' % (i, total_cross_entropy))
    print('训练之后的参数值：')
    print('w1', sess.run(w1))
    print('w2', sess.run(w2))

训练之前参数值：
w1: Tensor("Variable_10/read:0", shape=(2, 3), dtype=float32)
w2: Tensor("Variable_11/read:0", shape=(3, 1), dtype=float32)
After 0 trainnig step(s), croos entropy on all data is 0.0674925
After 1000 trainnig step(s), croos entropy on all data is 0.0163385
After 2000 trainnig step(s), croos entropy on all data is 0.00907547
After 3000 trainnig step(s), croos entropy on all data is 0.00714436
After 4000 trainnig step(s), croos entropy on all data is 0.00578471
训练之后的参数值：
w1 [[-1.9618274  2.582354   1.6820378]
 [-3.4681716  1.0698233  2.11789  ]]
w2 [[-1.8247149]
 [ 2.6854665]
 [ 1.418195 ]]
